# pyMOR Architecture

pyMOR's architecture is centered around ***generic*** implementations of model order reduction algorithms.

Algorithms are formulated in terms of ***abstract interfaces*** such that external PDE/NLA backends can easily be integrated.

## Motivation

Consider PDE solvers
- FEniCS
- deal.II
- ...
- proprietary code?

with specific backend for discretized operators and numerical linear algebra operations.

#### Can we implement MOR algorithms once in a way such that they are compatible with any desired PDE backend?

For the most part, yes! We only need notions of
- ***Operators***
- (Collections of) Vectors $\rightarrow$ ***VectorArrays***

and possible operations that involve these objects.

## `VectorArrays`

- Represent a (short) sequence/array of (high-dimensional) vectors.
  - sequences in Python implement `__getitem__` and `__len__`
  - arrays in NumPy implement advanced indexing
- Can represent a tall-and-skinny or short-and-fat matrix.
- Each vector (and thus the entire collection) lives in an associated `VectorSpace`.
- `VectorArrays` are never instantiated directly, but rather through a `VectorSpace`.
- Vectors in pyMOR are just `VectorArrays` of length one.
- pyMOR uses inheritence:
  - `pymor.vectorarrays.interface.VectorArray` defines the `VectorArray` interface
  - all concrete vector array classes need to inherit from `VectorArray`

In [ ]:
from pymor.basic import NumpyVectorSpace

space = NumpyVectorSpace(10)
print(space.dim)

In [ ]:
V = space.zeros(3)
print(V.dim)
print(len(V))
print(V)

In [ ]:
V = space.random(2)
print(V.dim)
print(len(V))
print(V)

In [ ]:
V = space.ones(1)
print(V.dim)
print(len(V))
print(V)

In [ ]:
import numpy as np

a = np.arange(30).reshape(10, 3)
print(a)

In [ ]:
V = space.make_array(a.T)
print(V.dim)
print(len(V))
print(V)

Internally, `VectorArrays` appear to be stored as row vectors, but

- `VectorArrays` in pyMOR are not considered ***column or row vectors***, they are simply lists of vectors.
- Mathematically there is no notion of row/column vectors from e.g. $\mathbb{R}^n$ either.
- Consequence: There is no `transpose` method for `VectorArrays`.
- Only typical operations for "mathematical" vectors are supported.

In [ ]:
V = space.make_array(np.arange(20).reshape(10, 2).T)
print(V)

W = space.make_array(np.arange(0, 40, 2).reshape(10, 2).T)
print(W)

In [ ]:
print(V + W)

In [ ]:
print(V - W)

In [ ]:
print(V * 2)

In [ ]:
print(V.norm())

In [ ]:
VW = V.copy()
VW.append(W)
print(VW)

### `VectorArray` vs NumPy array

Both types of arrays are used extensively in pyMOR.

`VectorArrays` are used whenever vectors that live in a (high-dimensional) state-space are represented, e.g.:

- internal state of a `Model`,
- basis used for projection,
- vectors that "live" in an external PDE solver.

NumPy arrays are used for (low-dimensional) data which has no direct connection to an external solver, e.g.:

- inputs and outputs of models,
- coefficients for linear combinations,
- computed inner products.

In [ ]:
print(V.inner(W))

In [ ]:
V.lincomb(np.array([1, 2]))

`VectorArrays` are ***mutable***.

Also, depending on the implementation, they may change the underlying data.
This is the case for `NumpyVectorArrays`.

In [ ]:
a = np.array([[1, 2, 3],
              [4, 5, 6]])
V = NumpyVectorSpace.make_array(a)
V.scal(2)
a

Depending on the `VectorArray` implementation, `to_numpy` and `from_numpy` methods may be available.

For `NumpyVectorArrays` this clearly works and `from_numpy` and `make_array` essentially do the same thing.

In [ ]:
V = NumpyVectorSpace.from_numpy(a)
V.to_numpy()

If you are using only NumPy-based data in your own application, using these methods is typically fine.

However, pyMOR code will generally avoid these methods since they may not be available for all `VectorArray` implementations.

Indexing and advanced indexing similar as for NumPy arrays is available and will always create a ***view on the data***.

In [ ]:
print(V)

In [ ]:
w = V[-1]
print(w)

In [ ]:
w.scal(2)

In [ ]:
print(w)

In [ ]:
print(V)

However, accessing individual entries via indexing is done via the `dofs` method.

In [ ]:
print(a[1, 1])
print(V[1].dofs([1]))

This is again motivated by the fact that the `VectorArray` may be part of a backend where extracting individual entries is either not possible or expensive.

Importantly, only a small number of `dofs` should be extracted at once.

### `ListVectorArray`

Linear algebra backends of many PDE solvers only have the notion of a single vector.

pyMOR supports an interface for single vectors via the `Vector` class.

A `VectorArray` can then simply be obtained via the `ListVectorArray` class which can be considerd a collection of corresponding `Vector` instances.

This means for a corresponding PDE solver backend one only needs to specify how a vector behaves by implementing a `*PDESolver*Vector` class and then gets the `VectorArray` for free.

In [ ]:
from pymor.vectorarrays.list import NumpyListVectorSpace

space = NumpyListVectorSpace(4)
W = space.random(3)
W.impl._list

### Exercise 1

Compute the Frobenius norm of `V.to_numpy()` without using `to_numpy`.

In [ ]:
V = NumpyVectorSpace.make_array(np.arange(20).reshape(10, 2).T)
np.linalg.norm(V.to_numpy())

In [ ]:
# your code here

### Exercise 2

Compute two linear combinations using a single call of `lincomb`.

In [ ]:
print(V.lincomb(np.array([1, 2])))
print(V.lincomb(np.array([3, 4])))

In [ ]:
print(...)  # your code here

### Exercise 3

Generate a normally distributed vector with 5 components of mean $1$ and standard deviation $0.1$.

In [ ]:
# your code here

## `Operators`

- `Operators` in pyMOR define a (non)parametric, (non)linear mapping between `VectorSpaces`.
- This means an `Operator` can be applied to any given `VectorArray` assuming that it is in the `Operators` source `VectorSpace`.
- Similar to `VectorArrays`, any concrete operator class needs to inherit from `pymor.operators.interface.Operator`.

In [ ]:
import scipy.sparse as sps

A = sps.diags([-2, 1, 1], [0, -1, 1], shape=(10, 10))

A.toarray()

In [ ]:
from pymor.operators.numpy import NumpyMatrixOperator

Aop = NumpyMatrixOperator(A)

print(Aop.range)
print(Aop.source)
print(Aop)
print(repr(Aop))

We can now create `VectorArrays` directly from the `VectorSpace` given by `Operator.source` and apply the `Operator`.

In [ ]:
V = Aop.source.ones(1)
W = Aop.source.zeros(1)
V.append(W)

Aop.apply(V)

`Operators` also have an `apply_inverse` method.

In [ ]:
Aop.apply_inverse(V)

Here, `Aop` is a `NumpyMatrixOperator` which can handle both dense NumPy arrays and sparse SciPy matrices.

The `apply_inverse` method chooses the appropriate solution approach for the corresponding data. E.g.,

- dense solver for dense NumPy arrays.
- SciPy sparse solvers for sparse SciPy matrices.

There are many options that can be passed to the `apply_inverse` method of a `NumpyMatrixOperator` to specifiy different solvers.

The easiest way to specify different solvers is via pyMORs `set_defaults` method.

In [ ]:
from pymor.core.defaults import set_defaults

set_defaults({'pymor.bindings.scipy.apply_inverse.default_solver': 'scipy_bicgstab_spilu'})

Now calling `NumpyMatrixOperator.apply_inverse` with a sparse SciPy matrix will use the specified solver without having to manually pass the argument.

Under the hood this is realized via the `@defaults` decorator.

In [ ]:
from pymor.core.defaults import defaults

@defaults('tolerance')
def some_algorithm(x, y, tolerance=1e-5):
    print(tolerance)

def test_some_algorithm(x, y, tolerance_for_some_algorithm=None):
    some_algorithm(x, y, tolerance=tolerance_for_some_algorithm)

In [ ]:
test_some_algorithm(1, 2)

Another option is using the `solver_options` parameter when initializing the `Operator`, but that specifies the solver only for the single `Operator`.

Generally, `Operators` might not have a specific `apply_inverse` method provided by the PDE backend.

Or the `Operator` is non-linear in which case one typically resorts to iterative methods.

In this case, pyMOR has its own generic iterative solver which only requires the `apply` method to be implemented.

In [ ]:
from pymor.core.defaults import *

set_defaults({
    'pymor.operators.numpy.NumpyMatrixOperator.apply_inverse.default_sparse_solver_backend': 'generic'
})

Aop.apply_inverse(V)

`Operators` also support various other methods.

In [ ]:
# Evaluate W.T @ A @ V
print(Aop.apply2(W, V))

In [ ]:
print(Aop.apply_adjoint(V))

In [ ]:
print(Aop.jacobian(V))

### `LincombOperator`

In MOR algorithms we may be interested to perform arithmetic operations with `Operators` in particular ***linear combinations*** of `Operators`.

Unlike for `VectorArrays`, adding, subtracting or scaling is not performed immediately but rather a `LincombOperator` is created.

In [ ]:
Bop = NumpyMatrixOperator(np.eye(10))
Lop = 3*Aop - 2*Bop
Lop

In [ ]:
Lop.apply(V)

In [ ]:
Lop.assemble()

Aside from the `LincombOperator` there are many different types of operators in `pymor.operators.constructions`.

In [ ]:
from pymor.operators.constructions import ZeroOperator, IdentityOperator, ConcatenationOperator

Zop = ZeroOperator(Aop.range, Aop.source)
Iop = IdentityOperator(Aop.range, Aop.source)
Cop = ConcatenationOperator((Aop, Bop))

LLop = Zop + Iop + Lop
LLop

In [ ]:
LLop.apply(V)

In [ ]:
LLop.assemble()

How does the `LincombOperator` know how to assemble the linear combination?

In [ ]:
from pymor.algorithms.lincomb import assemble_lincomb

assemble_lincomb?

In [ ]:
from pymor.algorithms.lincomb import AssembleLincombRules

AssembleLincombRules?

In [ ]:
AssembleLincombRules

In [ ]:
from pymor.algorithms.rules import RuleTable

RuleTable?

#### `projection` of `Operators`

Most important method of most model reduction algorithms is projection.

In [ ]:
from pymor.algorithms.projection import project

V = LLop.source.random(3)
W = LLop.range.random(3)

projected_LLop = project(LLop, W, V)
projected_LLop

Similar as for the `assemble` method of the `LincombOperator` we have a `RuleTable` that handles projection in a smart way.

pyMORs `Operators` can be implemented to be compatible with a desired PDE backend.

But also enable efficient and convenient handling of certain structured operators `BlockOperator`, `CanonicalSymplecticFormOperator`, `NumpyHankelOperator`, ...

Note that `Operators` in pyMOR are immutable.

Unlike for `VectorArrays`, we can not make internal changes to an `Operator`.

In [ ]:
# The code below will result in an error
# Aop.matrix = np.zeros((3, 3))

### Exercise

Convert an `Operator` to a matrix using `pymor.algorithms.to_matrix.to_matrix`.

In [ ]:
from pymor.algorithms.to_matrix import to_matrix

# your code here

## `Models`

- `Models` are collections of `Operators` and `VectorArrays` that represent a particular type of model.
- `pymor.models.interface.Model` defines the `solve` and `output` methods for the parameter-to-solution and parameter-output mapping, respectively.
  - This might be restrictive, e.g., a model might only have output and no state.
  - Thus, it is not necessary for a model to inherit from `Model`.
  - (An updated model hierarchy is a work in progress.)
- Model order reduction algorithms typically take `Models` as an input and compute a ROM which will again be a `Model`.

In [ ]:
from pymor.basic import *

p = StationaryProblem(
    domain=RectDomain([[0., 0.], [1., 1.]], left='robin', right='robin', top='robin', bottom='robin'),
    diffusion=ConstantFunction(1., 2),
    robin_data=(ConstantFunction(1., 2), ExpressionFunction('(x[0] < 1e-10) * 1.', 2)),
    outputs=[('l2_boundary', ExpressionFunction('(x[0] > (1 - 1e-10)) * 1.', 2))]
)

fom, _ = discretize_stationary_cg(p, diameter=0.1)

In [ ]:
fom

In [ ]:
fom.rhs

In [ ]:
V = fom.solve()

In [ ]:
fom.visualize(V)

In [ ]:
from pymor.models.transfer_function import TransferFunction

tf = TransferFunction(1, 1, lambda s: 1 / (s + 1))

tf.eval_tf(1j)

In pyMOR `Models` are immutable objects.

The `with_` method creates a deep copy of an underlying object and allows for passing additional input arguments for the class.

In [ ]:
# The code below will result in error
# fom.name = 'SimpleProblem'

fom.with_(name='SimpleProblem')

Typically, evaluating large-scale models is very expensive.

In order to prevent expensive computations to be performed multiple times, pyMOR has its own caching mechanism.

The easiest way to enable caching in pyMOR is to use the `@cached` decorator on a method of a model which inherits from `CacheableObject`.

In [ ]:
from pymor.core.cache import CacheableObject, cached

In [ ]:
class CustomModel(CacheableObject):

    cache_region = 'memory'

    @cached
    def expensive_evaluation(self, x):
        ...

Aside from `'memory'` one can also use `'disk'` or `'persistent'` as cache regions or configure a custom cache region.

## Parameters and Parametric Objects

`Operators` and `Models` can generally be parametrized by one or more parameters in pyMOR.

### `Parameters`

- Mapping from parameter names to dimensions.
- Defines what parameters a parametric object can accept.
- Names `'t'`, `'s'`, and `'input'` should only be used for time, frequency, and input, respectively.

In [ ]:
from pymor.parameters.base import Parameters

parameters = Parameters({'a': 1, 'b': 2})
parameters

`Parameters` are immutable.

In [ ]:
parameters['c'] = 3

### `Mu`

- Mapping from parameters names to 1D NumPy arrays.
- Represents parameter values.

In [ ]:
from pymor.parameters.base import Mu

mu = Mu({'a': np.array([1]), 'b': np.array([2, 3]), 'c': np.array([4, 5, 6])})
mu

It is possible to check if parameter values belong to a `Parameter`.

In [ ]:
parameters.is_compatible(mu)

It is possible to avoid constructing `Mu` by hand if `Parameters` is available via its `parse` method.

In [ ]:
parameters.parse([1, 2, 3])

### `ParameterSpace`

- Represents the set of possible parameter values.
- Restricted to be of cubic type (each parameter component has a minimum and maximum value).

In [ ]:
from pymor.parameters.base import ParameterSpace

print(ParameterSpace(parameters, (0, 1)))
print(ParameterSpace(parameters, {'a': (0, 1), 'b': (1, 2)}))

A `ParameterSpace` can also be created using the `space` method of `Parameters`.

In [ ]:
print(parameters.space(0, 1))
print(parameters.space({'a': (0, 1), 'b': (1, 2)}))

`ParameterSpaces` are useful for sampling.

In [ ]:
parameter_space = parameters.space(0, 1)
parameter_space.sample_uniformly(3)

In [ ]:
parameter_space.sample_randomly(10)

### `ParameterFunctional`

- Represent mappings from parameter values to $\mathbb{R}$ or $\mathbb{C}$.

In [ ]:
from pymor.parameters.functionals import ProjectionParameterFunctional

f1 = ProjectionParameterFunctional('b', size=2, index=0)
f1.evaluate(mu=parameters.parse([1, 2, 3]))

In [ ]:
from pymor.parameters.functionals import GenericParameterFunctional

f2 = GenericParameterFunctional(lambda mu: mu['b'][0]**2, parameters)
f2.evaluate(mu=parameters.parse([1, 2, 3]))

In [ ]:
from pymor.parameters.functionals import ExpressionParameterFunctional

f3 = ExpressionParameterFunctional('b[0]**2', parameters)
f3.evaluate(mu=parameters.parse([1, 2, 3]))

### Time-Dependent Parameter Values

In [ ]:
mu = Parameters(f=1, t=1).parse(['sin(t)', 0])
mu

In [ ]:
mu['f']

In [ ]:
mu2 = mu.with_(t=np.pi/4)
mu2

In [ ]:
mu2['f']

### Parametric `Operators`

The simplest case is a parameter-separable `Operator`.

In [ ]:
Pop = Aop + ProjectionParameterFunctional('p') * Bop
Pop

It is possible to check if an `Operator` (or any object inheriting from `ParametricObject`) is parametric via its `parametric` attribute.

In [ ]:
Pop.parametric

The `Parameters` of the `Operator` are automatically deduced.

In [ ]:
Pop.parameters

We can `apply` the `Operator` on a `VectorArray` for different parameter values.

In [ ]:
V = Pop.source.ones(1)
Pop.apply(V, mu=Pop.parameters.parse(0))

In [ ]:
Pop.apply(V, mu=Pop.parameters.parse(1))

Note that it is necessary to pass a `Mu` object.

In [ ]:
Pop.apply(V, mu=1)

`VectorArrays` can not be parametric.
A way to represent parameter-dependent vectors is via a parametric `Operator`,
e.g., using `VectorArrayOperators`.

In [ ]:
from pymor.operators.constructions import VectorArrayOperator

space = NumpyVectorSpace(10)

U = space.ones(1)
print(U)
V = space.ones(1)
print(V)

In [ ]:
Uop = VectorArrayOperator(U)
print(Uop)
Vop = VectorArrayOperator(V)
print(Vop)

In [ ]:
UVop = Uop + ProjectionParameterFunctional('p') * Vop
print(UVop)

In [ ]:
UVop.as_vector(mu=UVop.parameters.parse(5))

## Parametric Models

If a `Model` is based on any parametric `Operators`, the `Model` will be parametric as well.

In [ ]:
A = np.random.rand(10, 10)
B = np.ones((10, 1))
C = np.ones((1, 10))
fom = LTIModel.from_matrices(A, B, C)

fom

In [ ]:
fom.parameters

In [ ]:
Pop = fom.A + ProjectionParameterFunctional('p') * fom.A
fom = fom.with_(A=Pop)

In [ ]:
fom

In [ ]:
fom.parameters

## Conclusion

- Fundamental interfaces:
  - `VectorArray`
  - `Operator`
  - `Model`
- Extensively used features:
  - defaults
  - caching
  - parameters
  - rule tables